# Plotting: histograms

In this notebook, we illustrate the possibilities of plotting 1D and 2D histograms.

Note that Osyris's plotting functions are wrapping Matplotlib's plotting functions,
and forwards most Matplotlib arguments to the underlying function.

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

path = "osyrisdata/starformation"
data = osyris.Dataset(8, scale="au", path=path).load()

## 1D histogram of gas density

The `histogram1d` function provides a simple way to quickly make 1D histogram plots:

In [ ]:
osyris.histogram1d(data["hydro"]["density"], logx=True)

The bin edges can be specified using the `bins` parameter,
which can either be an integer number or an array (similarly to Numpy's `bins` argument):

In [ ]:
osyris.histogram1d(data["hydro"]["density"], logx=True,
                   bins=np.logspace(-18., -13., 10),
                   color='m')

## Multiple 1D histograms

Multiple histograms can be over-plotted on the same axes by passing multiple layers:

In [ ]:
bins = np.linspace(-0.15, 0.15, 40)
osyris.histogram1d({"data": data["hydro"]["B_field"].x, "alpha": 0.5},
                   {"data": data["hydro"]["B_field"].y, "alpha": 0.5},
                   {"data": data["hydro"]["B_field"].z, "alpha": 0.5},
                   logy=True, bins=bins)

It is also possible to specify different bins for different layers:

In [ ]:
osyris.histogram1d({"data": data["hydro"]["B_field"].x, "alpha": 0.5, "bins": 40},
                   {"data": data["hydro"]["B_field"].y, "alpha": 0.5, "bins": 10},
                   logy=True)

## A 2D histogram of density vs magnetic field

By default, the `histogram2d` function will show a binned count of cells

In [ ]:
osyris.histogram2d(data["hydro"]["density"], data["hydro"]["B_field"],
                   norm="log", loglog=True)

Additional components can be overlayed using `layers`:

In [ ]:
osyris.histogram2d(
    data["hydro"]["density"], data["hydro"]["B_field"],
    {"data": data["hydro"]["mass"], "norm": "log"}, # layer 1
    {"data": data["amr"]["level"], "operation": "mean",
     "mode": "contour", "colors": "k"}, # layer 2
    loglog=True)

## Controlling the resolution

By default, the histograms and the maps have a resolution of 256x256 pixels.
This can be changed via the keyword argument `resolution`,
which can either be an integer (in which case the same resolution is applied to both the x and y dimension)
or a dict.

In [ ]:
# Create figure
fig, ax = plt.subplots(1, 2, figsize=(12, 4.5))

osyris.histogram2d(data["hydro"]["density"], data["hydro"]["B_field"],
                   norm="log", loglog=True, ax=ax[0],
                   resolution=64)

osyris.map({"data": data["hydro"]["density"], "norm": "log"},
           dx=2000 * osyris.units("au"),
           dy=1000 * osyris.units("au"),
           origin=center, direction="z", ax=ax[1],
           resolution={'x': 512, 'y': 256})

ax[1].set_aspect("equal")

## Tiled plots

In this example, we create two subplot axes with Matplotlib.

Next, we plot in the left panel the log of density as a coloured slice with velocity vectors.
The minimum and maximum of $\log(\rho)$ are forced to `-17` and `-11`.
We give the `map` call the axes to use via the `ax` argument.
Next, we overlay some custom chosen density contours with different line styles and colours.

In the right panel, we plot a plane of temperature and overlay some lightgray contours showing the AMR levels.
We specify only integer contour levels.

In [ ]:
# Create figure
fig, ax = plt.subplots(1, 2, figsize=(12, 4.5))

# Define region to plot
dx = 2000.0 * osyris.units("au")

# Left plot: coloured density slice with overlayed contours
osyris.map({"data": data["hydro"]["density"], "norm": "log", "vmin": 1.0e-17, "vmax": 1.0e-11},
           {"data": data["hydro"]["velocity"], "mode": "vec"},
           {"data": data["hydro"]["density"], "mode": "contour",
            "levels": [1.0e-17,1.0e-16,1.0e-12], "colors": ('yellow','k',"lime"),
            "linewidths": [2,5,2], "linestyles": ["solid","dashed","solid"],
            "cmap": None, "labels": False},
           dx=dx, origin=center, direction="z", ax=ax[0])

osyris.map({"data": data["hydro"]["temperature"], "norm": "log", "mode": "contourf",
            "levels": np.logspace(0.9, 2, 11), "cmap": "hot"},
           {"data": data["amr"]["level"], "mode": "contour", "colors": "w", "levels": [6, 7, 8]},
           dx=dx, origin=center, direction="z", ax=ax[1])

## Difference between two snapshots

Here, we want to make a map of the difference in density between two snapshots.
Because we do not necessarily have the same number of cells at the same place,
we first have to make uniform 2D maps using the `map` function,
which we can then directly compare.

The `map` function actually returns an object that contains the raw data that was used to create the figure.
By using the `plot=False` argument, we can silence the figure generation, and use the data in a custom figure.

**Note:** For this to make sense, the two outputs have to be centered around the same center point.

In [ ]:
# Read data from an earlier snapshot
data_old = osyris.Dataset(5, scale="au", path=path).load()

## Scatter plot

Make a scatter plot of density vs temperature, with only 1 in 1000 cells.
Colour the dots according to the magnitude of the gas velocity.

In [ ]:
step = 100
osyris.scatter(data["hydro"]["density"][::step],
               data["hydro"]["velocity"][::step],
               c=data["hydro"]["thermal_pressure"][::step],
               norm="log", loglog=True)